# Capstone Project
## Image classifier for the SVHN dataset
### Instructions

In this notebook, you will create a neural network that classifies real-world images digits. You will use concepts from throughout this course in building, training, testing, validating and saving your Tensorflow classifier model.

This project is peer-assessed. Within this notebook you will find instructions in each section for how to complete the project. Pay close attention to the instructions as the peer review will be carried out according to a grading rubric that checks key parts of the project instructions. Feel free to add extra cells into the notebook as required.

### How to submit

When you have completed the Capstone project notebook, you will submit a pdf of the notebook for peer review. First ensure that the notebook has been fully executed from beginning to end, and all of the cell outputs are visible. This is important, as the grading rubric depends on the reviewer being able to view the outputs of your notebook. Save the notebook as a pdf (File -> Download as -> PDF via LaTeX). You should then submit this pdf for review.

### Let's get started!

We'll start by running some imports, and loading the dataset. For this project you are free to make further imports throughout the notebook as you wish. 

In [ ]:
import tensorflow as tf
from scipy.io import loadmat
import matplotlib.pyplot as plt
import numpy as np
import os
import random

from scipy.io import loadmat
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout

![SVHN overview image](data/svhn_examples.jpg)
For the capstone project, you will use the [SVHN dataset](http://ufldl.stanford.edu/housenumbers/). This is an  image dataset of over 600,000 digit images in all, and is a harder dataset than MNIST as the numbers appear in the context of natural scene images. SVHN is obtained from house numbers in Google Street View images. 

* Y. Netzer, T. Wang, A. Coates, A. Bissacco, B. Wu and A. Y. Ng. "Reading Digits in Natural Images with Unsupervised Feature Learning". NIPS Workshop on Deep Learning and Unsupervised Feature Learning, 2011.

Your goal is to develop an end-to-end workflow for building, training, validating, evaluating and saving a neural network that classifies a real-world image into one of ten classes.

In [ ]:
# Run this cell to load the dataset

train = loadmat('train_32x32.mat')
test = loadmat('test_32x32.mat')

Both `train` and `test` are dictionaries with keys `X` and `y` for the input images and labels respectively.

## 1. Inspect and preprocess the dataset
* Extract the training and testing images and labels separately from the train and test dictionaries loaded for you.
* Select a random sample of images and corresponding labels from the dataset (at least 10), and display them in a figure.
* Convert the training and test images to grayscale by taking the average across all colour channels for each pixel. _Hint: retain the channel dimension, which will now have size 1._
* Select a random sample of the grayscale images and corresponding labels from the dataset (at least 10), and display them in a figure.

In [ ]:
X_train = train['X']
X_test = test['X']
y_train = train['y']
y_test = test['y']

In [ ]:
X_train = np.moveaxis(X_train, -1, 0)
X_test = np.moveaxis(X_test, -1 , 0)

In [ ]:
y_train = np.where(y_train==10, 0, y_train)
y_test = np.where(y_test==10, 0, y_test)

In [ ]:
indices = random.sample( range(0,X_train.shape[0]), 10 )
fig, ax = plt.subplots(1, 10, figsize=(15,1))

for i in range(10):
    ax[i].set_axis_off()
    ax[i].imshow(X_train[indices[i]])
    ax[i].set_title(y_train[indices[i]])

In [ ]:
X_train_greyscale = np.mean(X_train, -1, keepdims=True)
X_test_greyscale = np.mean(X_test, -1, keepdims=True)

In [ ]:
indices = random.sample( range(0,X_train_greyscale.shape[0]), 10 )
fig, ax = plt.subplots(1, 10, figsize=(15,1))

for i in range(10):
    ax[i].set_axis_off()
    ax[i].imshow(X_train_greyscale[indices[i],:,:,0], cmap='gray')
    ax[i].set_title(y_train[indices[i]])

## 2. MLP neural network classifier
* Build an MLP classifier model using the Sequential API. Your model should use only Flatten and Dense layers, with the final layer having a 10-way softmax output. 
* You should design and build the model yourself. Feel free to experiment with different MLP architectures. _Hint: to achieve a reasonable accuracy you won't need to use more than 4 or 5 layers._
* Print out the model summary (using the summary() method)
* Compile and train the model (we recommend a maximum of 30 epochs), making use of both training and validation sets during the training run. 
* Your model should track at least one appropriate metric, and use at least two callbacks during training, one of which should be a ModelCheckpoint callback.
* As a guide, you should aim to achieve a final categorical cross entropy training loss of less than 1.0 (the validation loss might be higher).
* Plot the learning curves for loss vs epoch and accuracy vs epoch for both training and validation sets.
* Compute and display the loss and accuracy of the trained model on the test set.

In [ ]:
def get_MLP_model():
    
    Model = Sequential([
        Flatten(input_shape=X_train[0].shape),
        Dense(1024, activation='relu'),
        Dense(512, activation='relu'),
        Dense(256, activation='relu'),
        Dense(128, activation='relu'),
        Dense(10, activation='softmax')
    ])
    
    return(Model)

model = get_MLP_model()
model.summary()

In [ ]:
checkpoint_path = 'Checkpoint/best_model'
checkpoint = ModelCheckpoint(checkpoint_path,
                     save_best_only=True,
                     save_weights_only=True,
                     verbose=2,
                     save_freq='epoch',
                     monitor='val_accuracy',
                     mode='max')

In [ ]:
EarlyStop = EarlyStopping(monitor='val_accuracy', mode='max', patience=3)

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=30,
                    batch_size=64, verbose=1,
                    validation_split=0.15,
                    callbacks=[checkpoint, EarlyStop])

In [ ]:
def plot_metrics(h):
    fig, axs = plt.subplots(1,2, figsize=(12, 6))
    axs[0].plot(h['loss'], label='loss')
    axs[0].plot(h['val_loss'], label='val_loss')
    axs[0].legend()
    
    axs[1].plot(h['accuracy'], label='accuracy')
    axs[1].plot(h['val_accuracy'], label='val_accuracy')
    axs[1].legend()

plot_metrics(history.history)

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=False)
print(f"Test Loss is {test_loss}")
print(f"Test Accuracy is {test_accuracy}")

## 3. CNN neural network classifier
* Build a CNN classifier model using the Sequential API. Your model should use the Conv2D, MaxPool2D, BatchNormalization, Flatten, Dense and Dropout layers. The final layer should again have a 10-way softmax output. 
* You should design and build the model yourself. Feel free to experiment with different CNN architectures. _Hint: to achieve a reasonable accuracy you won't need to use more than 2 or 3 convolutional layers and 2 fully connected layers.)_
* The CNN model should use fewer trainable parameters than your MLP model.
* Compile and train the model (we recommend a maximum of 30 epochs), making use of both training and validation sets during the training run.
* Your model should track at least one appropriate metric, and use at least two callbacks during training, one of which should be a ModelCheckpoint callback.
* You should aim to beat the MLP model performance with fewer parameters!
* Plot the learning curves for loss vs epoch and accuracy vs epoch for both training and validation sets.
* Compute and display the loss and accuracy of the trained model on the test set.

In [ ]:
def get_CNN_model(input_shape):
    
    Model = Sequential([
        Conv2D(16, (3,3), padding='same', activation='relu', input_shape=input_shape),
        BatchNormalization(),
        MaxPooling2D((2,2)),
        Conv2D(16, (3,3), padding='same', activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2,2)),
        Conv2D(16, (3,3), padding='same', activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2,2)),
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.2),
        Dense(128, activation='relu'),
        Dropout(0.2),
        Dense(10, activation='softmax')
    ])
    
    return(Model)

CNN_model = get_CNN_model(X_train[0].shape)
CNN_model.summary()

In [ ]:
checkpoint_path_CNN = 'Checkpoint/best_model_CNN'
checkpoint_CNN = ModelCheckpoint(checkpoint_path_CNN,
                     save_best_only=True,
                     save_weights_only=True,
                     verbose=2,
                     save_freq='epoch',
                     monitor='val_accuracy',
                     mode='max')

In [ ]:
EarlyStop_CNN = EarlyStopping(monitor='val_accuracy', mode='max', patience=3)

In [ ]:
CNN_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history_CNN = CNN_model.fit(X_train, y_train, epochs=30,
                    batch_size=64, verbose=1,
                    validation_split=0.15,
                    callbacks=[checkpoint_CNN, EarlyStop_CNN])

In [ ]:
def plot_metrics(h):
    fig, axs = plt.subplots(1,2, figsize=(12, 6))
    axs[0].plot(h['loss'], label='loss')
    axs[0].plot(h['val_loss'], label='val_loss')
    axs[0].legend()
    
    axs[1].plot(h['accuracy'], label='accuracy')
    axs[1].plot(h['val_accuracy'], label='val_accuracy')
    axs[1].legend()
    
plot_metrics(history_CNN.history)

In [ ]:
test_loss_CNN, test_accuracy_CNN = CNN_model.evaluate(X_test, y_test, verbose=False)
print(f"Test Loss is {test_loss_CNN}")
print(f"Test Accuracy is {test_accuracy_CNN}")

## 4. Get model predictions
* Load the best weights for the MLP and CNN models that you saved during the training run.
* Randomly select 5 images and corresponding labels from the test set and display the images with their labels.
* Alongside the image and label, show each model’s predictive distribution as a bar chart, and the final model prediction given by the label with maximum probability.

In [ ]:
Best_MLP = get_MLP_model()
Best_MLP.load_weights('Checkpoint/best_model')

In [ ]:
Best_CNN = get_CNN_model(X_train[0].shape)
Best_CNN.load_weights('Checkpoint/best_model_CNN')

In [ ]:
def show_predictive_distribution(model):

    num_test_images = X_test.shape[0]

    random_inx = np.random.choice(num_test_images, 5)
    random_test_images = X_test[random_inx, ...]
    random_test_labels = y_test[random_inx, ...]

    predictions = model.predict(random_test_images)

    fig, axes = plt.subplots(5, 2, figsize=(16, 12))
    fig.subplots_adjust(hspace=0.4, wspace=-0.2)

    for i, (prediction, image, label) in enumerate(zip(predictions, random_test_images, random_test_labels)):
        axes[i, 0].imshow(np.squeeze(image))
        axes[i, 0].get_xaxis().set_visible(False)
        axes[i, 0].get_yaxis().set_visible(False)
        axes[i, 0].text(10., -1.5, f'Digit {label}')
        axes[i, 1].bar(np.arange(len(prediction)), prediction)
        axes[i, 1].set_xticks(np.arange(len(prediction)))
        axes[i, 1].set_title(f"Categorical distribution. Model prediction: {np.argmax(prediction)}")
    
    plt.show()

In [ ]:
show_predictive_distribution(Best_MLP)

In [ ]:
show_predictive_distribution(Best_CNN)